In [1]:
import nltk
from nltk.corpus import reuters
import re, pprint
from nltk import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from six import string_types
from nltk.corpus import wordnet as wn
import unicodedata
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd

In [2]:

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yogsy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yogsy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
blg_df = pd.read_csv('C:/Users/yogsy/Desktop/PG Course/Project 10 - Statistical NLP Project/blogtext.csv/blogtext.csv')

In [4]:
blg_df.shape

(681284, 7)

In [5]:
blg_df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [8]:
#selecting only 7500 line items

In [6]:
blg_df=blg_df[0:7500]

In [8]:
#preprocessing of data

In [7]:
blg_df=blg_df.drop(['id','date'],axis=1)

In [9]:
blg_df.head()

,gender,age,topic,sign,text
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,..."
1,male,15,Student,Leo,These are the team members: Drewe...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing!!! testing!!!
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...


In [10]:
#Remove accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [12]:
blg_df['text']

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
5                    I had an interesting conversation...
6                    Somehow Coca-Cola has a way of su...
7                    If anything, Korea is a country o...
8                    Take a read of this news article ...
9                    I surf the English news sites a l...
10                   Ah, the Korean language...it look...
11                   If you click on my profile you'll...
12                   Last night was pretty fun...mostl...
13                   There is so much that is differen...
14                    urlLink    Here it is, the super...
15                   One thing I love about Seoul (and...
16                    urlLink    Wonderful oh-gyup-sal...
17            

In [13]:
#Remove unwanted characters
def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]' 
    text = re.sub(pattern, '', text)
    return text

In [14]:
remove_special_characters("Thanks to Yahoo!'s Toolbar I can", remove_digits=True)

'Thanks to Yahoos Toolbar I can'

In [15]:
#Convert text to lowercase
blg_df['text_new']=blg_df['text'].str.lower()

In [16]:
## Text lemmatization
def lemmatize_text(text):

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])  

In [17]:
lemmatize_text("take me home with you forever where I")

'take me home with you forever where I'

In [18]:
## Text stemming

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text



In [19]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [20]:
def normalize_corpus(corpus, html_stripping=True, accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [21]:
blg_df['text_new'] =normalize_corpus(blg_df['text_new'])

In [22]:
blg_df.head()

,gender,age,topic,sign,text,text_new
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info ha been found pages and mb of pdf files n...
1,male,15,Student,Leo,These are the team members: Drewe...,these are the team members drewes van der laag...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...
3,male,15,Student,Leo,testing!!! testing!!!,testing testing
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...


In [23]:
blg_df.shape

(7500, 6)

In [24]:
##Q3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence

In [25]:
blg_df['label'] =blg_df["gender"].map(str)+ ', ' +blg_df["age"].map(str)+ ', ' +blg_df["topic"].map(str)+ ', ' +blg_df["sign"]

In [26]:
blg_df.head()

,gender,age,topic,sign,text,text_new,label
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info ha been found pages and mb of pdf files n...,"male, 15, Student, Leo"
1,male,15,Student,Leo,These are the team members: Drewe...,these are the team members drewes van der laag...,"male, 15, Student, Leo"
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...,"male, 15, Student, Leo"
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"male, 15, Student, Leo"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...,"male, 33, InvestmentBanking, Aquarius"


In [27]:
blgdata=blg_df[['text_new','label']]

In [28]:
blgdata.head()

,text_new,label
0,info ha been found pages and mb of pdf files n...,"male, 15, Student, Leo"
1,these are the team members drewes van der laag...,"male, 15, Student, Leo"
2,in het kader van kernfusie op aarde maak je ei...,"male, 15, Student, Leo"
3,testing testing,"male, 15, Student, Leo"
4,thanks to yahoo s toolbar i can now capture th...,"male, 33, InvestmentBanking, Aquarius"


In [29]:
blgdata['label'].nunique()

114

In [30]:
blgdata.astype

<bound method NDFrame.astype of                                                text_new  \
0     info ha been found pages and mb of pdf files n...   
1     these are the team members drewes van der laag...   
2     in het kader van kernfusie op aarde maak je ei...   
3                                      testing testing    
4     thanks to yahoo s toolbar i can now capture th...   
5     i had an interesting conversation with my dad ...   
6     somehow cocacola ha a way of summing up thing ...   
7     if anything korea is a country of extremes eve...   
8     take a read of this news article from urllink ...   
9     i surf the english news site a lot looking for...   
10    ah the korean language it look so difficult at...   
11    if you click on my profile youll make a notsos...   
12    last night wa pretty fun mostly because of the...   
13    there is so much that is different here from a...   
14    urllink here it is the superfantastic phonebox...   
15    one thing i love a

In [31]:
##Separate features and labels, and split the data into training and testing

In [32]:
X=blgdata['text_new']
Y=blgdata['label']
print(X.shape)
print(Y.shape)

(7500,)
(7500,)


In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=2)

In [35]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5625,) (5625,)
(1875,) (1875,)


In [36]:
##Vectorize the features
##Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label.

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 2),stop_words='english')


In [38]:
cv.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [39]:
len(cv.vocabulary_)

334387

In [40]:
cv.vocabulary_

{'urllink': 305274,
 'princess': 223607,
 'bride': 30892,
 'character': 41555,
 'quiz': 228977,
 'wa': 311094,
 'mysti': 190985,
 'cheated': 42130,
 'result': 239498,
 'turn': 300841,
 'buttercup': 34293,
 'just': 149920,
 'yeah': 330887,
 'working': 327432,
 'btw': 32298,
 'hate': 123885,
 'quizzes': 229034,
 'stumble': 275629,
 'em': 82853,
 'figure': 96772,
 'id': 135933,
 'share': 255202,
 'cause': 39277,
 'thats': 286046,
 'urllink urllink': 306658,
 'urllink princess': 306306,
 'princess bride': 223608,
 'bride character': 30894,
 'character quiz': 41626,
 'quiz wa': 229005,
 'wa urllink': 313673,
 'urllink mysti': 306155,
 'mysti cheated': 190986,
 'cheated result': 42140,
 'result turn': 239563,
 'turn wa': 301062,
 'wa buttercup': 311405,
 'buttercup just': 34294,
 'just yeah': 151291,
 'yeah working': 331130,
 'working btw': 327445,
 'btw hate': 32315,
 'hate quizzes': 124068,
 'quizzes just': 229037,
 'just stumble': 151100,
 'stumble em': 275634,
 'em figure': 82871,
 'figu

In [41]:
x_train_dtm=cv.transform(X_train)

In [42]:
x_test_dtm=cv.transform(X_test)

In [43]:
x_train_dtm[0]

<1x334387 sparse matrix of type '<class 'numpy.int64'>'
	with 51 stored elements in Compressed Sparse Row format>

In [44]:
x_test_dtm[0]

<1x334387 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [45]:
print(x_train_dtm.shape)
print(x_test_dtm.shape)

(5625, 334387)
(1875, 334387)


In [46]:
x_train_dtm

<5625x334387 sparse matrix of type '<class 'numpy.int64'>'
	with 671849 stored elements in Compressed Sparse Row format>

In [47]:
x_test_dtm

<1875x334387 sparse matrix of type '<class 'numpy.int64'>'
	with 120755 stored elements in Compressed Sparse Row format>

In [48]:
print('Features: ', x_train_dtm.shape[1])

Features:  334387


In [49]:
print('Features: ', x_test_dtm.shape[1])

Features:  334387


In [50]:
print(blgdata)

                                               text_new  \
0     info ha been found pages and mb of pdf files n...   
1     these are the team members drewes van der laag...   
2     in het kader van kernfusie op aarde maak je ei...   
3                                      testing testing    
4     thanks to yahoo s toolbar i can now capture th...   
5     i had an interesting conversation with my dad ...   
6     somehow cocacola ha a way of summing up thing ...   
7     if anything korea is a country of extremes eve...   
8     take a read of this news article from urllink ...   
9     i surf the english news site a lot looking for...   
10    ah the korean language it look so difficult at...   
11    if you click on my profile youll make a notsos...   
12    last night wa pretty fun mostly because of the...   
13    there is so much that is different here from a...   
14    urllink here it is the superfantastic phonebox...   
15    one thing i love about seoul and i mean this a... 

In [51]:
pd.value_counts(blgdata['label'])

male, 35, Technology, Aries                2294
male, 36, Fashion, Aries                   1200
female, 27, indUnk, Taurus                  605
female, 17, indUnk, Scorpio                 558
female, 34, indUnk, Sagittarius             532
female, 24, indUnk, Scorpio                 191
female, 17, indUnk, Cancer                  139
female, 15, Student, Libra                  131
male, 24, Engineering, Libra                119
female, 27, Education, Aquarius              85
male, 17, Sports-Recreation, Capricorn       75
male, 33, InvestmentBanking, Aquarius        70
female, 15, Student, Aquarius                62
female, 36, indUnk, Pisces                   59
male, 23, indUnk, Sagittarius                49
male, 39, Communications-Media, Libra        47
male, 25, Non-Profit, Cancer                 46
male, 14, Student, Scorpio                   46
male, 26, indUnk, Gemini                     44
male, 14, indUnk, Scorpio                    42
male, 14, Student, Leo                  

In [52]:
blgdata.groupby('label').count()

,text_new
label,
"female, 13, indUnk, Libra",9
"female, 14, Student, Capricorn",2
"female, 14, indUnk, Aquarius",5
"female, 14, indUnk, Aries",21
"female, 14, indUnk, Cancer",7
"female, 14, indUnk, Leo",4
"female, 15, Arts, Pisces",2
"female, 15, Student, Aquarius",62
"female, 15, Student, Cancer",9


In [53]:
##Transform the labels

In [54]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

In [55]:
mlb = MultiLabelBinarizer(classes=Y_train.unique())
mlb.fit_transform('Y_train')
mlb.transform('Y_test')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:951: UserWarning: unknown class(es) ['Y', '_', 'a', 'i', 'n', 'r', 't'] will be ignored
  .format(sorted(unknown, key=str)))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:951: UserWarning: unknown class(es) ['Y', '_', 'e', 's', 't'] will be ignored
  .format(sorted(unknown, key=str)))


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        

In [56]:
x_train_dtm.shape

(5625, 334387)

In [57]:
print(Y_train.shape)
print(Y_test.shape)

(5625,)
(1875,)


In [58]:
##Choose a classifier

In [59]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',max_iter=2000)
clf1vR = OneVsRestClassifier(clf)

In [60]:
##Fit the classifier, make predictions and get the accuracy

In [61]:
clf1vR.fit(x_train_dtm,Y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=2000,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [62]:
predicted=clf1vR.predict(x_test_dtm)

In [72]:
from sklearn.metrics import accuracy_score
print( "Accuracy Score: ",accuracy_score(Y_test, predicted))

Accuracy Score:  0.632


In [73]:
print(Y_train,predicted)

6316                 male, 36, Fashion, Aries
3134              male, 35, Technology, Aries
4536          female, 34, indUnk, Sagittarius
84                  female, 14, indUnk, Aries
5072              female, 17, indUnk, Scorpio
1811              male, 35, Technology, Aries
7076                 male, 36, Fashion, Aries
2152              male, 35, Technology, Aries
2963              male, 35, Technology, Aries
6419                 male, 36, Fashion, Aries
5929               female, 27, indUnk, Taurus
5938               female, 27, indUnk, Taurus
5683               female, 27, indUnk, Taurus
5451               female, 27, indUnk, Taurus
2832              male, 35, Technology, Aries
383               female, 24, indUnk, Scorpio
1892              male, 35, Technology, Aries
1172               female, 16, indUnk, Taurus
7215                 male, 36, Fashion, Aries
4297          female, 34, indUnk, Sagittarius
192              female, 37, indUnk, Aquarius
7097                 male, 36, Fas

In [74]:
nb = MultinomialNB()
nb.fit(x_train_dtm, Y_train)
y_pred_class = nb.predict(x_test_dtm)

In [75]:
print (metrics.accuracy_score(Y_test, y_pred_class))

0.4512


In [76]:
logreg = LogisticRegression()
logreg.fit(x_train_dtm, Y_train)
y_pred_class = logreg.predict(x_test_dtm)
print (metrics.accuracy_score(Y_test, y_pred_class))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6330666666666667


In [84]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(X_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, Y_train)
    y_pred_class = nb.predict(x_train_dtm)
    print('Accuracy: ', metrics.accuracy_score(Y_test, y_pred_class))

In [86]:
x_train_dtm = vect.fit_transform(X_train)
x_test_dtm = vect.transform(X_test)


In [89]:
vect = CountVectorizer(stop_words='english', max_features=300)
tokenize_test(vect) ## need help with below error

Features:  300


ValueError: Found input variables with inconsistent numbers of samples: [1875, 5625]